In [3]:
# Import
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [36]:
# Parameters
n = 50
C = 4
T = 30
M = 20
sT = 3
# The number of items in a customer's cart is an EXPONENTIAL random variable

# Factors
p = 0.5
K = 8

# CSV name
csv_name = "VTMC4S3"

In [6]:
CSV = pd.read_csv(f"{csv_name}.csv").fillna(0)
CSV = CSV[CSV["type"] == "scalar"]

# NQ: Mean Number of Customers in Queue [Queues]
Exp_NQ = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_NQ = Exp_NQ[Exp_NQ["name"] == "queueLength:timeavg"]

# RHO: Mean Utilization [Queues]
Exp_RHO = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_RHO = Exp_RHO[Exp_RHO["name"] == "busy:timeavg"]

# S: Mean Service Time (t_S) [LocalSink] 
Exp_S = CSV[CSV["module"].str.contains("SuperMarket.checkout.localSink")]
Exp_S = Exp_S[Exp_S["name"] == "totalServiceTime:mean"]

# W: Mean Waiting Time [Queues]
Exp_W = CSV[CSV["module"].str.contains("SuperMarket.checkout.queues")]
Exp_W = Exp_W[Exp_W["name"] == "queueingTime:mean"]


# Dictionary for containing all the DataFrames
CSV_Dict = {
    "NQ": Exp_NQ,
    "RHO": Exp_RHO,
    "S": Exp_S,
    "W": Exp_W
}


PI = {}

Q = []
for i in range(C):
    Q.append(f"Q{i}")

for key in CSV_Dict:
    PI[key] = pd.DataFrame(columns=Q)
    for i in range(C):
        temp = []
        temp = [CSV_Dict[key]["value"].values[j*C + i] for j in range(n)]
        string = f"Q{i}"
        temp = list(map(float, temp))
        PI[key][string] = temp
    


In [28]:
PI_TABLE = {}

for key in PI:
    PI_TABLE[key] = pd.DataFrame(columns=["Theoretical", "Mean", "CI - LB", "CI - UB"])
    
    # Mean Value of the "n" repetitions for each Queue
    PI_TABLE[key]["Mean"] = PI[key].mean()

    # CI 95%
    # Lower Bound
    PI_TABLE[key]["CI - LB"] = PI[key].mean() - (1.96 * (PI[key].std() / math.sqrt(n)))
    # Upper Bound
    PI_TABLE[key]["CI - UB"] = PI[key].mean() + (1.96 * (PI[key].std() / math.sqrt(n)))



In [29]:
for key in PI_TABLE:
    print(f"[{key}]:")
    print(PI_TABLE[key])
    print("\n")

[NQ]:
   Theoretical      Mean   CI - LB   CI - UB
Q0         NaN  0.003109  0.002982  0.003235
Q1         NaN  0.003242  0.003124  0.003360
Q2         NaN  0.332645  0.324153  0.341137
Q3         NaN  0.333185  0.323529  0.342842


[RHO]:
   Theoretical      Mean   CI - LB   CI - UB
Q0         NaN  0.068238  0.067737  0.068740
Q1         NaN  0.068516  0.068004  0.069028
Q2         NaN  0.495311  0.492740  0.497881
Q3         NaN  0.493090  0.490103  0.496077


[S]:
   Theoretical       Mean    CI - LB    CI - UB
Q0         NaN  13.191007  13.149797  13.232216
Q1         NaN  13.177710  13.139199  13.216221
Q2         NaN  67.779758  67.545005  68.014511
Q3         NaN  67.543779  67.288179  67.799379


[W]:
   Theoretical       Mean    CI - LB    CI - UB
Q0         NaN   0.600101   0.577663   0.622539
Q1         NaN   0.623169   0.601910   0.644428
Q2         NaN  45.482173  44.417600  46.546746
Q3         NaN  45.589726  44.388771  46.790681




In [30]:
PI_TABLE["S"]["Theoretical"] = [1, 2, 3, 4] 

In [31]:
PI_TABLE["S"]

,Theoretical,Mean,CI - LB,CI - UB
Q0,1,13.191007,13.149797,13.232216
Q1,2,13.177710,13.139199,13.216221
Q2,3,67.779758,67.545005,68.014511
Q3,4,67.543779,67.288179,67.799379


In [34]:
temp = []
N = 2
temp = [N] * 3
M = 5
temp = temp + ([M] * 2)
temp

[2, 2, 2, 5, 5]

In [41]:
# Theoretical Formulas and Computations

# Number of Quick Tills
quickTills = int(p*C)

# Interarrival Rate
t = 1/T

temp = {}

if (quickTills == 0) or quickTills == C:
    # There are no Quick Tills or all the Tills are Quick Tills
    # So the probability alphaK to being routed to a Quick Till is 0 or 1

    pi_ = 1/C
    t_ = t*pi_

    S_ = M*sT
    s_ = 1/S_

    rho_ = t_/s_
    N_ = rho_/(1-rho_)
    NQ_ = N_ - rho_
    R_ = N_/t_
    W_ = R_ - S_

    temp["NQ"] = NQ_
    temp["RHO"] = rho_
    temp["S"] = S_
    temp["W"] = W_

    for key in PI_TABLE:
        PI_TABLE[key]["Theoretical"] = [temp[key]] * C

else:
    # There are Quick Tills and Normal Tills
    # So the probability alphaK to being routed to a Quick Till is 0 < alphaK < 1
    
    # CDF of an EXPONENTIAL distribution
    # with lambda = 1/(M-1) and x = K-0.5
    # F(x; lambda) = alphaK = 1 - e^(-lambda*x)
    alphaK = 1 - math.exp(-(K-0.5)/(M-1))
    
    # Probability of being routed to a specific Quick Till (i)
    # Inside a subsystem (Quick Tills or Normal Tills) the
    # probability of being routed to a specific Till is the
    # same (i.e. "equallylikely")
    pi_i = alphaK/quickTills
    
    # Interarrival Rate for an individual Quick Till
    t_i = t*pi_i
    
    # Service Time for an individual Quick Till
    # 
    S_i = ((quad(integrand, 0, K-0.5, limit=200)[0])/alphaK)*sT
    s_i = 1/S_i

    # Variance = E[X^2] - E[X]^2
    
    VarS_i = ((quad(integrand_2, 0, K-0.5, limit=200)[0])/alphaK)*sT**2 - S_i**2


    
    
    ...

    


In [ ]:
Ma alphaK a dividere è per normalizzare il tutto?
Provare a calcolare integrale di distribuzione troncata senza calcolare la media sugli stessi intervalli e vedere se fa 1

In [ ]:
# C=4 DISCRETE
# This Code uses PI_T from the above cell
from scipy.integrate import quad
import math
import numpy as np

C = 4
p = 0.5
T = 40
t = 1/T
M = 15
K = 8
sT = 3
alphaK = 1 - math.exp(-(K-0.5)/(M-1))

def integrand(x):
    return (math.floor((x+1+0.5))*(1/(M-1))*math.exp(-x/(M-1)))

def integrand_2(x):
    return ((((math.floor(x+1+0.5))**2))*(1/(M-1))*math.exp(-x/(M-1)))


pi_i = alphaK/(C*p)
t_i = t*pi_i

S_i = ((quad(integrand, 0, K-0.5, limit=200)[0])/alphaK)*sT
s_i = 1/S_i

Var_S_i = ((quad(integrand_2, 0, K-0.5, limit=200)[0])/alphaK)*sT**2 - S_i**2

rho_i = t_i/s_i
# N_i = rho_i/(1-rho_i)
N_i = rho_i + ((rho_i**2 + t_i**2 * Var_S_i)/(2*(1-rho_i)))
NQ_i = N_i - rho_i
R_i = N_i/t_i
W_i = R_i - S_i


pi_j = (1-alphaK)/(C*(1-p))
t_j = t*pi_j

S_j = ((quad(integrand, K-0.5, np.inf, limit=200)[0])/(1-alphaK))*sT
s_j = 1/S_j

Var_S_j = ((quad(integrand_2, K-0.5, np.inf, limit=200)[0])/(1-alphaK))*sT**2 - S_j**2

rho_j = t_j/s_j
# N_j = rho_j/(1-rho_j)
N_j = rho_j + ((rho_j**2 + t_j**2 * Var_S_j)/(2*(1-rho_j)))
NQ_j = N_j - rho_j
R_j = N_j/t_j
W_j = R_j - S_j




if "Theoretical" not in PI_T["RHO"].columns:
    PI_T["RHO"].insert(0, "Theoretical", [rho_i, rho_i, rho_j, rho_j])
else:
    PI_T["RHO"]["Theoretical"] = [rho_i, rho_i, rho_j, rho_j]

if "Theoretical" not in PI_T["S"].columns:
    PI_T["S"].insert(0, "Theoretical", [S_i, S_i, S_j, S_j])
else:
    PI_T["S"]["Theoretical"] = [S_i, S_i, S_j, S_j]

if "Theoretical" not in PI_T["NQ"].columns:
    PI_T["NQ"].insert(0, "Theoretical", [NQ_i, NQ_i, NQ_j, NQ_j])
else:
    PI_T["NQ"]["Theoretical"] = [NQ_i, NQ_i, NQ_j, NQ_j]

if "Theoretical" not in PI_T["W"].columns:
    PI_T["W"].insert(0, "Theoretical", [W_i, W_i, W_j, W_j])
else:
    PI_T["W"]["Theoretical"] = [W_i, W_i, W_j, W_j]

for key in PI_T:
    PI_T[key]["OK"] = [""] * 4
    for i in range(4):
        if PI_T[key]["Theoretical"].iloc[i] <= PI_T[key]["CI (UB)"].iloc[i] and PI_T[key]["Theoretical"].iloc[i] >= PI_T[key]["CI (LB)"].iloc[i]:
            PI_T[key].loc[f"Q{i}", "OK"] = "V"
        else:
            PI_T[key].loc[f"Q{i}", "OK"] = ""

for key in PI_T:
    print(f"    [{key}]")
    print(PI_T[key])
    print("\n")


In [ ]:
# DOUBLE numOfItems
# This Code uses PI_T from the above cell
from scipy.integrate import quad
import math
import numpy as np

C = 4
p = 0.5
T = 20
t = 1/T
M = 10
K = 8
alphaK = 1 - math.exp(-K/M)

def integrand(x):
    return ((x)*(1/M)*math.exp(-x/M))


pi_i = alphaK/(C*p)
t_i = t*pi_i

S_i = round(((quad(integrand, 0, K)[0])/alphaK)*3, 6)
s_i = 1/S_i

rho_i = t_i/s_i
N_i = rho_i/(1-rho_i)
NQ_i = N_i - rho_i
R_i = N_i/t_i
W_i = R_i - S_i


pi_j = (1-alphaK)/(C*(1-p))
t_j = t*pi_j

S_j = round(((quad(integrand, K, np.inf)[0])/(1-alphaK))*3, 6)
s_j = 1/S_j

rho_j = t_j/s_j
N_j = rho_j/(1-rho_j)
NQ_j = N_j - rho_j
R_j = N_j/t_j
W_j = R_j - S_j



if "Theoretical" not in PI_T["RHO"].columns:
    PI_T["RHO"].insert(0, "Theoretical", [rho_i, rho_i, rho_j, rho_j])
else:
    PI_T["RHO"]["Theoretical"] = [rho_i, rho_i, rho_j, rho_j]

if "Theoretical" not in PI_T["S"].columns:
    PI_T["S"].insert(0, "Theoretical", [S_i, S_i, S_j, S_j])
else:
    PI_T["S"]["Theoretical"] = [S_i, S_i, S_j, S_j]

if "Theoretical" not in PI_T["NQ"].columns:
    PI_T["NQ"].insert(0, "Theoretical", [NQ_i, NQ_i, NQ_j, NQ_j])
else:
    PI_T["NQ"]["Theoretical"] = [NQ_i, NQ_i, NQ_j, NQ_j]

if "Theoretical" not in PI_T["W"].columns:
    PI_T["W"].insert(0, "Theoretical", [W_i, W_i, W_j, W_j])
else:
    PI_T["W"]["Theoretical"] = [W_i, W_i, W_j, W_j]

for key in PI_T:
    PI_T[key]["OK"] = [""] * 4
    for i in range(4):
        if PI_T[key]["Theoretical"].iloc[i] <= PI_T[key]["CI (UB)"].iloc[i] and PI_T[key]["Theoretical"].iloc[i] >= PI_T[key]["CI (LB)"].iloc[i]:
            PI_T[key].loc[f"Q{i}", "OK"] = "V"
        else:
            PI_T[key].loc[f"Q{i}", "OK"] = ""

for key in PI_T:
    print(f"    [{key}]")
    print(PI_T[key])
    print("\n")


In [ ]:
# We add the PI value of the queue q to the DataFrame
# at the row 'run' and at the column 'q'
PI_df[pi][config].loc[int(run), q] = PI[pi][config][run][q]

In [ ]:
# C = 1
# This Code uses PI_T from the above cell

C = 1
T = 20  
t = 1/T
M = 4


t_j = t

S_j = M*3
s_j = 1/S_j

rho_j = t_j/s_j
N_j = rho_j/(1-rho_j)
NQ_j = N_j - rho_j
R_j = N_j/t_j
W_j = R_j - S_j



if "Theoretical" not in PI_T["RHO"].columns:
    PI_T["RHO"].insert(0, "Theoretical", [rho_j])
else:
    PI_T["RHO"]["Theoretical"] = [rho_j]

if "Theoretical" not in PI_T["S"].columns:
    PI_T["S"].insert(0, "Theoretical", [S_j])
else:
    PI_T["S"]["Theoretical"] = [S_j]

if "Theoretical" not in PI_T["NQ"].columns:
    PI_T["NQ"].insert(0, "Theoretical", [NQ_j])
else:
    PI_T["NQ"]["Theoretical"] = [NQ_j]

if "Theoretical" not in PI_T["W"].columns:
    PI_T["W"].insert(0, "Theoretical", [W_j])
else:
    PI_T["W"]["Theoretical"] = [W_j]

for key in PI_T:
    PI_T[key]["OK"] = [""] * 1
    for i in range(1):
        if PI_T[key]["Theoretical"].iloc[i] <= PI_T[key]["CI (UB)"].iloc[i] and PI_T[key]["Theoretical"].iloc[i] >= PI_T[key]["CI (LB)"].iloc[i]:
            PI_T[key].loc[f"Q{i}", "OK"] = "V"
        else:
            PI_T[key].loc[f"Q{i}", "OK"] = ""

for key in PI_T:
    print(f"    [{key}]")
    print(PI_T[key])
    print("\n")
